# LangChain 시작하기

## 01. 랭체인(LangChain) 개요

### 📌 개요
- **LangChain** = LLM을 실용적으로 사용하기 위한 프레임워크
- 목적:
  1. **LLM + 외부 데이터** (DB, 문서, API 등) 연결
  2. **LLM + 도구** (검색, 계산기, 코드 실행) 연결
  3. **LLM 워크플로우** 관리 및 체인화

### ⚙️ 주요 구성 요소
| 구성 요소 | 설명 |
|-----------|------|
| **PromptTemplate** | 프롬프트를 템플릿화하여 재사용 가능 |
| **LLM** | GPT, Claude, LLaMA 등 다양한 모델 연결 |
| **Chains** | 여러 단계를 묶어 순차적으로 실행 |
| **Agents** | LLM이 도구(검색, 계산기, DB 등)를 선택·실행 |
| **Memory** | 대화 기록 및 컨텍스트 유지 |


---

## 02. LangChain

### 📌 개념
- LangChain은 **OpenAI, Anthropic, HuggingFace** 등 다양한 LLM 연결 지원
- `ChatOpenAI` 클래스로 대화형 모델과 직접 통신
- **temperature**: 창의성 조절 (0 = 정확성↑, 1 = 다양성↑)  
- **model**: 모델 종류 선택 (예: `gpt-3.5-turbo`, `gpt-4o-mini`)  

In [1]:
### 💻 예시 코드
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
print(api_key[:10]) 

sk-proj-73


In [2]:
from langchain_openai import ChatOpenAI

# 모델 초기화
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0.7
)

# 질문 보내기
response = llm.invoke("LangChain이 무엇인지 간단히 설명해줘")
print(response.content)

LangChain은 언어 간 상호작용을 촉진하기 위해 설계된 블록체인 기술입니다. 이 기술은 다양한 언어간 커뮤니케이션을 용이하게 하고, 어휘, 문법, 문화 차이를 극복하는데 도움을 줍니다. LangChain은 전 세계 사람들이 서로 소통하고 협력할 수 있는 환경을 제공하는 것을 목표로 합니다.


## 03. 토큰(Token), 토큰계산기, 모델별 비용

### 📌 개념

- 토큰(Token) = 언어모델이 처리하는 최소 단위 (sub-word, 음절 단위)
- 토큰 수 = 입력 + 출력 합산 (→ 비용과 속도에 영향)
- Context Window = 모델이 한 번에 처리 가능한 최대 토큰 수
- 비용 = 모델별 1K tokens 기준 요금 존재 (예: GPT-3.5 = $0.0015)

## 04. ChatOpenAI 주요 파라미터, invoke(), stream() 

### 📌 주요 파라미터
- **model** : 사용할 모델 이름 (예: `gpt-3.5-turbo`, `gpt-4o`)
- **temperature** : 창의성 조절 (0 → 사실 위주, 1 → 창의적)
- **max_tokens** : 출력 최대 토큰 수
- **top_p, top_k** : 샘플링 방식 조절

In [4]:
### 💻 예시 코드
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)

# invoke() → 한번에 응답 생성
response = llm.invoke("LangChain의 핵심 기능을 3줄로 요약해줘.")
print(response.content)

print("----------------------------------------------")

# stream() → 스트리밍 응답
for chunk in llm.stream("실시간으로 답변을 생성해줘."):
    print(chunk.content, end="")

LangChain은 언어 번역 및 통역을 제공하는 플랫폼으로, 사용자들은 다양한 언어 간의 소통을 쉽게 할 수 있습니다. 또한 블록체인 기술을 활용하여 보안성과 신뢰성을 높이며, 지속적인 언어 학습과 발전을 지원합니다. 또한 사용자들은 LangChain 내에서 다양한 언어 관련 서비스를 이용할 수 있습니다.
----------------------------------------------
알겠습니다! 무엇을 도와드릴까요?

## 05. LangSmith 로 GPT 추론내용 추적
### 📌 개념

- LangSmith = LangChain 공식 실험/관찰 플랫폼
- LLM 추론 과정(입력, 출력, 체인 실행 로그) 기록 가능

In [6]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# 모델 초기화
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 프롬프트 템플릿
prompt = PromptTemplate.from_template("다음 문장을 영어로 번역해줘: {sentence}")

# 체인 생성 (LCEL)
chain = prompt | llm

# 실행 (LangSmith 대시보드에 자동 기록됨)
result = chain.invoke({"sentence": "안녕하세요, 오늘 날씨가 참 좋네요."})
print(result.content)

Hello, the weather is really nice today.


In [7]:
from langchain.schema import StrOutputParser

parser_chain = prompt | llm | StrOutputParser()

result = parser_chain.invoke({"sentence": "오늘 저녁에 영화 보러 가자."})
print(result)  # 문자열만 출력

Let's go watch a movie tonight.


In [8]:
from langchain.schema.runnable import RunnableParallel

multi_chain = RunnableParallel(
    english=(PromptTemplate.from_template("Translate to English: {text}") | llm | StrOutputParser()),
    french=(PromptTemplate.from_template("Translate to French: {text}") | llm | StrOutputParser())
)

result = multi_chain.invoke({"text": "나는 오늘 아침에 커피를 마셨다."})
print(result)

{'english': 'I drank coffee this morning.', 'french': 'Je ai bu du café ce matin.'}


## 06. GPT-4o (멀티모달) 모델로 이미지 인식하여 답변 출력
### 📌 개념

- GPT-4o = OpenAI 멀티모달 모델
- 텍스트 + 이미지 입력 가능

In [14]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model="gpt-4o")

message = HumanMessage(
    content=[
        {"type": "text", "text": "이 이미지에는 무엇이 보이나요?"},
        {
            "type": "image_url",
            "image_url": {
                "url": "https://picsum.photos/200"
            }
        }
    ]
)

response = llm.invoke([message])
print(response.content)

이미지에는 "MOTEL"이라는 글자가 쓰인 간판이 보입니다. 간판은 기둥에 세워져 있고 하늘을 배경으로 하고 있습니다. 상단에는 안테나 같은 구조물이 보입니다.


## 07. LCEL 로 Chain 생성 
### 📌 개념

- LCEL (LangChain Expression Language)
- 체인을 직관적으로 | 연산자로 연결 가능

In [10]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template("Translate the following into Korean: {text}")
llm = ChatOpenAI(model="gpt-3.5-turbo")

chain = prompt | llm
response = chain.invoke({"text": "Good morning"})
print(response.content)

좋은 아침 (joh-eun achim)


- 여러 입력을 동시에 처리 (batch inference)

In [12]:
inputs = [{"text": "Hello"}, {"text": "Thank you"}]
results = chain.batch(inputs)
print([r for r in results])

['안녕하세요', '감사합니다 (gam-sa-ham-ni-da)']


## 08. 출력파서(StrOutputParser) 를 체인에 연결 
### 📌 개념

- 모델 출력 → 문자열로 변환
- 여러 후처리 파서 제공 (JSON, List 등)

In [11]:
from langchain.schema import StrOutputParser

chain = prompt | llm | StrOutputParser()
print(chain.invoke({"text": "Nice to meet you"}))

반가워요


## 09. 비동기(asynchronous) 호출 방법 
### 📌 개념

- asyncio 기반 비동기 실행
- 동시에 여러 요청 처리 가능

In [19]:
# import asyncio

# async def run_async():
#     result = await chain.ainvoke({"text": "Good night"})
#     print(result)

# asyncio.run(run_async())

In [15]:
result = await chain.ainvoke({"text": "Good night"})
print(result)

잘 자세요 (jal jaseyo)


## 10. 병렬체인 구성(RunnableParallel)
### 📌 개념

- 여러 체인을 병렬로 실행 후 결과 합치기

In [16]:
from langchain.schema.runnable import RunnableParallel

parallel_chain = RunnableParallel(
    korean=prompt | llm | StrOutputParser(),
    english=prompt | llm | StrOutputParser()
)

results = parallel_chain.invoke({"text": "How are you?"})
print(results)

{'korean': '"How are you?" in Korean is "잘 지내세요?" (jal jinaeseyo?).', 'english': '"How are you?" can be translated into Korean as "어떻게 지내세요?"'}


## 11. 값을 전달해주는 RunnablePassthrough
### 📌 개념

- 입력을 그대로 다음 체인에 전달

In [17]:
from langchain.schema.runnable import RunnablePassthrough

passthrough_chain = RunnablePassthrough() | llm
response = passthrough_chain.invoke("그냥 이 문장을 모델로 보내줘.")
print(response.content)

물론입니다! 원하시는 문장을 말씀해주시면 그대로 전달해드리겠습니다.


## 12. 병렬로 Runnable 을 실행하는 RunnableParallel
### 📌 개념

- 여러 Runnable을 동시에 실행
- 결과를 dict 형태로 반환

## 13. 함수를 실행하는 RunnableLambda, itemgetter
### 📌 개념

- RunnableLambda : Python 함수 실행을 Runnable로 래핑
- itemgetter : 입력 dict에서 특정 key만 추출

In [18]:
from langchain.schema.runnable import RunnableLambda
from operator import itemgetter

def to_upper(x):
    return x.upper()

lambda_chain = RunnableLambda(to_upper) | llm
print(lambda_chain.invoke("hello"))

# itemgetter 예시
getter_chain = itemgetter("text") | llm
print(getter_chain.invoke({"text": "LangChain is fun"}).content)

content='Hello! How can I assist you today?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 9, 'total_tokens': 18, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_80956533cb', 'id': 'chatcmpl-C9aMg2lxJptqDHcmFXXHAb5Hf1gwK', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--df4b1473-ed8e-4b5e-8e3c-b1548cc6b6cc-0' usage_metadata={'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
I'm glad to hear that you're enjoying LangChain! It's a popular framework for building applications with large language models. If you have any questions or need more information about how t

## ✅ 정리

- ChatOpenAI 주요 메서드: invoke(), stream(), batch(), ainvoke()

- LCEL 문법으로 체인을 직관적으로 구성 가능 (|)

- LangSmith에서 체인 실행과정을 모니터링 가능

- Runnable 계열 (RunnableParallel, RunnablePassthrough, RunnableLambda)로 유연한 체인 설계 가능